https://thefiddler.substack.com/p/how-many-rides-can-you-reserve

----

My plan is to calculate the expected number of rides by using some recursive functions.

I am going to label the slots 1 to 12 for convenience. 1 is (9am-10am) and 12 is (8pm - 9pm).

----

In [43]:
from functools import cache

@cache
def additional_rides(existing_slots=frozenset(), max_slot=12, debug=0, policy='Fiddler'):
    """
    :param existing_slots: frozenset of existing slots
    :param max_slot: Maximum slot number
    :return: Expected number of rides 
    """
    
    sum_EV = 0
    num_EV = 0
    
    max_existing_slot = max(existing_slots, default=0)
    min_existing_slot = min(existing_slots, default=0)

    if (len(existing_slots) < 3):
        start_value = 1 
    else:
        if policy == 'Fiddler':
            start_value = max_existing_slot + 1
        elif policy == 'ExtraCredit':
            start_value = min_existing_slot + 1
        else:
            raise ValueError("Invalid policy. Use 'Fiddler' or 'ExtraCredit'.")
    
    for i in range(start_value, max_slot + 1):
        if i not in existing_slots:
            slots_new = existing_slots | {i}
            if (policy == 'ExtraCredit' and len(slots_new) > 3):
                slots_new = slots_new - {min_existing_slot}
            sum_EV += 1 + additional_rides(slots_new, max_slot, debug, policy)
            num_EV += 1
        
    avg_EV = (sum_EV / num_EV if num_EV > 0 else 0)

    if (debug > 0):
        print (f"existing_slots: {existing_slots}, max_slot: {max_slot}, sum_EV: {sum_EV}, num_EV: {num_EV}, avg_EV: {avg_EV}")
    
    return avg_EV


Checking simpler cases first.

In [44]:
assert(additional_rides(max_slot=3, debug=0) == 3)

----

And now we can solve the Fiddler and the Extra Credit

In [45]:
additional_rides(max_slot=12, debug=0, policy='Fiddler')

4.269877344877345

In [46]:
additional_rides(max_slot=12, debug=0, policy='ExtraCredit')

6.809632034632034

---

But to make sure, I am going to also solve the fiddler a different way.

Probability of particular max values after the first 3 are chosen can computed directly.

And expected additional turns given a max value can be computed iteratively.

In [47]:
from fractions import Fraction as F
# prob max == k is prob max is not 12, and prob max is not 11 ... down to not k+1, and is k.
@cache
def p_max(k, max_slot=12):
    if k < 3 or max_slot < k:
        return 0
    p = 1
    for i in reversed(range(k+1, max_slot + 1)):
        p *= F(i - 3, i)
    p *= F(3,k)
    return p

# Additional rides given that max so far is m is the average of the expected number of rides for each possible next max slot.
from statistics import mean
@cache
def more_rides(m, max_slot=12):
    if max_slot <= m:
        return F(0)
    s = mean( [ (F(1) + more_rides(i,max_slot)) for i in range(m+1, max_slot+1) ] )
    return s

In [48]:
pmX = [p_max(i) for i in range(1, 13)]
mrX = [more_rides(i) for i in range(1, 13)]

print("pX: ", pmX)
print("mX: ", mrX)

sum(pmX)

pX:  [0, 0, Fraction(1, 220), Fraction(3, 220), Fraction(3, 110), Fraction(1, 22), Fraction(3, 44), Fraction(21, 220), Fraction(7, 55), Fraction(9, 55), Fraction(9, 44), Fraction(1, 4)]
mX:  [Fraction(83711, 27720), Fraction(7381, 2520), Fraction(7129, 2520), Fraction(761, 280), Fraction(363, 140), Fraction(49, 20), Fraction(137, 60), Fraction(25, 12), Fraction(11, 6), Fraction(3, 2), Fraction(1, 1), Fraction(0, 1)]


Fraction(1, 1)

In [49]:
expected_value = sum([pmX[i] * (3 + mrX[i]) for i in range(0, 12)])
print("Expected value: ", expected_value, "   =  ", expected_value*1.0)

Expected value:  118361/27720    =   4.269877344877345


Okay, it matches !!!
I like this way of calculating much better, as it could almost be done by hand.


But not trying to solve the extra credit this way. :)

Honestly, it's because I am not too certain how to extend this scheme to the extra credit - I think that's because we need to track more information for the extra credit, so it seems harder to factor as cleanly.

----